'''
For open-set identification, we follow our previous work

It consists of two steps, enrollment and identification 

LFW, IJBC and VGG2

1.0 generate the deep features/ hash codes 
1.1 enrollement according to the protocol
1.2 identifcation 
2.0 performance evaluation

'''

In [1]:
from absl import app, flags, logging
from absl.flags import FLAGS
import os
import numpy as np
import tensorflow as tf
from modules.utils import set_memory_growth, load_yaml, l2_norm
from modules.models import ArcFaceModel, IoMFaceModelFromArFace, IoMFaceModelFromArFaceMLossHead,IoMFaceModelFromArFace2,IoMFaceModelFromArFace3,IoMFaceModelFromArFace_T,IoMFaceModelFromArFace_T1
import tqdm
import csv


In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

logger = tf.get_logger()
logger.disabled = True
logger.setLevel(logging.FATAL)
# set_memory_growth()

cfg = load_yaml('./configs/iom_res50_random.yaml')  # cfg = load_yaml(FLAGS.cfg_path)
permKey = None
if cfg['head_type'] == 'IoMHead':  #
    # permKey = generatePermKey(cfg['embd_shape'])
    permKey = tf.eye(cfg['embd_shape'])  # for training, we don't permutate, won't influence the performance

arcmodel = ArcFaceModel(size=cfg['input_size'],
                        embd_shape=cfg['embd_shape'],
                        backbone_type=cfg['backbone_type'],
                        head_type='ArcHead',
                        training=False,
                        cfg=cfg)

ckpt_path = tf.train.latest_checkpoint('./checkpoints/arc_res50')
if ckpt_path is not None:
    print("[*] load ckpt from {}".format(ckpt_path))
    arcmodel.load_weights(ckpt_path)
else:
    print("[*] Cannot find ckpt from {}.".format(ckpt_path))
    exit()
    



[*] load ckpt from ./checkpoints/arc_res50\e_17_b_21576.ckpt


In [3]:
def load_data_from_dir(save_path, BATCH_SIZE=128, img_ext='png'):
    def transform_test_images(img):
        img = tf.image.resize(img, (112, 112))
        img = img / 255
        return img

    def get_label_withname(file_path):
        # convert the path to a list of path components
        parts = tf.strings.split(file_path, os.path.sep)
        # The second to last is the class-directory
#         wh = tf.strings.split(parts[-1], ".")[0]
        wh = tf.strings.split(parts[-1], ".")[0]
        return wh

    def process_path_withname(file_path):
        label = get_label_withname(file_path)
        img = tf.io.read_file(file_path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = transform_test_images(img)
        return img, label

    list_gallery_ds = tf.data.Dataset.list_files(save_path + '/*/*.' + img_ext, shuffle=False)
    labeled_gallery_ds = list_gallery_ds.map(lambda x: process_path_withname(x))
    dataset = labeled_gallery_ds.batch(BATCH_SIZE)
    return dataset

def extractFeat(dataset, model, feature_dim):
    final_feature = np.zeros(feature_dim)
    feats = []
    names = []
    n = 0
    for image_batch, label_batch in tqdm.tqdm(dataset):
        feature = model(image_batch)
        for i in range(feature.shape[0]):
            n = n + 1
            feats.append(feature[i].numpy())
            mylabel = str(label_batch[i].numpy().decode("utf-8")+"")
#             print(mylabel)
            names.append(mylabel)
            
    return feats,names,n


In [5]:
dataset = load_data_from_dir('./data/lfw_mtcnnpy_160', BATCH_SIZE=128)
feats,names,n = extractFeat(dataset, arcmodel, 512)
with open('lfw_feat.csv', 'w') as f: 
    # using csv.writer method from CSV package 
    write = csv.writer(f) 
    write.writerows(feats) 
with open('lfw_name.txt', 'w') as outfile:
    for i in names:
        outfile.write(i+"\n")

0it [00:00, ?it/s]

    def get_label_withname(file_path):
        # convert the path to a list of path components
        parts = tf.strings.split(file_path, os.path.sep)
        # The second to last is the class-directory
#         wh = tf.strings.split(parts[-1], ".")[0]
        wh = tf.strings.split(parts[-1], ".")[0]
        return wh

This may be caused by multiline strings or comments not indented at the same level as the code.


104it [09:54,  5.71s/it]


In [8]:

m = cfg['m'] = 512
q = cfg['q'] = 2

# here I add the extra IoM layer and head
if cfg['hidden_layer_remark'] == '1':
    model = IoMFaceModelFromArFace(size=cfg['input_size'],
                                   arcmodel=arcmodel, training=False,
                                   permKey=permKey, cfg=cfg)
elif cfg['hidden_layer_remark'] == '2':
    model = IoMFaceModelFromArFace2(size=cfg['input_size'],
                                    arcmodel=arcmodel, training=False,
                                    permKey=permKey, cfg=cfg)
elif cfg['hidden_layer_remark'] == '3':
    model = IoMFaceModelFromArFace3(size=cfg['input_size'],
                                    arcmodel=arcmodel, training=False,
                                    permKey=permKey, cfg=cfg)
elif cfg['hidden_layer_remark'] == 'T':  # 2 layers
    model = IoMFaceModelFromArFace_T(size=cfg['input_size'],
                                     arcmodel=arcmodel, training=False,
                                     permKey=permKey, cfg=cfg)
elif cfg['hidden_layer_remark'] == 'T1':
    model = IoMFaceModelFromArFace_T1(size=cfg['input_size'],
                                      arcmodel=arcmodel, training=False,
                                      permKey=permKey, cfg=cfg)
else:
    model = IoMFaceModelFromArFace(size=cfg['input_size'],
                                   arcmodel=arcmodel, training=False,
                                   permKey=permKey, cfg=cfg)
model.summary(line_length=80)
cfg['embd_shape'] = m * q

Model: "IoMface_model"
________________________________________________________________________________
Layer (type)                        Output Shape                    Param #     
input_image (InputLayer)            [(None, 112, 112, 3)]           0           
________________________________________________________________________________
arcface_model (Model)               (None, 512)                     40375680    
________________________________________________________________________________
perm_layer_1 (PermLayer)            (None, 512)                     0           
________________________________________________________________________________
IoMProjectionLayer (Model)          (None, 1024)                    525312      
________________________________________________________________________________
IoMHead (Model)                     (None, 512)                     0           
Total params: 40,900,992
Trainable params: 40,842,752
Non-trainable params: 58,240
___

In [9]:
feats,names,n = extractFeat(dataset, model, m)
with open('lfw_feat_dIoM_512x2.csv', 'w') as f: 
    # using csv.writer method from CSV package 
    write = csv.writer(f) 
    write.writerows(feats) 

104it [10:32,  6.08s/it]
